In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_schema = "order_id long , order_date string, customer_id long,order_status string"

In [3]:
orders_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/orders/orders_1gb.csv")

In [4]:
customer_schema = " customerid long , customer_fname string , customer_lname string , user_name string,password string , address string, city string, state string, pincode long "

In [5]:
customers_df = spark.read \
.format("csv") \
.schema(customer_schema) \
.load("/retail_db/customers")

In [6]:
spark.conf.get('spark.sql.autoBroadcastJoinThreshold')

'10485760b'

In [7]:
orders_df.join(customers_df, orders_df.customer_id == customers_df.customerid, "inner").write.format("noop").mode("overwrite").save()

In [ ]:
orders_df.join(customers_df, orders_df.customer_id == customers_df.customerid, "inner").explain()

In [9]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold', '-1')

In [10]:
orders_df.join(customers_df, orders_df.customer_id == customers_df.customerid, "inner").write.format("noop").mode("overwrite").save()

In [ ]:
orders_df.join(customers_df, orders_df.customer_id == customers_df.customerid, "inner").explain()

In [12]:
orders_df.join(customers_df.hint("shuffle_hash"), orders_df.customer_id == customers_df.customerid, "inner").write.format("noop").mode("overwrite").save()

In [ ]:
orders_df.join(customers_df.hint("shuffle_hash"), orders_df.customer_id == customers_df.customerid, "inner").explain()